In [60]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from scipy.optimize import fmin_l_bfgs_b
import time
from PIL import Image

tf.compat.v1.disable_eager_execution() #must put at the TOP

from tensorflow.keras.preprocessing.image import load_img, img_to_array

# This is the path to the image you want to extract texture from
target_image_path = 'C:/Users/MrLin/Documents/Experiments/Fine_Scale_Autoencoder_Lichen/Results/test_images/input/New folder/05.png'

# Dimensions of the generated picture.
img_height = 512 #set intended image size to be given to load_img below
img_width = 512

def process_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img_array = img_to_array(img)
    img_array *= (1/255)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def deprocess_image(img_tensor):
    img_tensor *= 255
    img_tensor = np.clip(img_tensor, 0, 255)
    img_tensor = np.uint8(img_tensor)
    return img_tensor

target_image = K.constant(process_image(target_image_path))
learned_image = K.placeholder((1, img_height, img_width, 3))

#new input that we will attach to the model that will be involved in the image learning algorithm
input_tensor = K.concatenate([target_image,
                              learned_image], axis=0)
print(target_image)
print(learned_image)
print(input_tensor)

Tensor("Const_164:0", shape=(1, 512, 512, 3), dtype=float32)
Tensor("Placeholder_8:0", shape=(1, 512, 512, 3), dtype=float32)
Tensor("concat_22:0", shape=(2, 512, 512, 3), dtype=float32)


In [62]:
# Rebuild model with a the new input tensor
new_input = Input(tensor=input_tensor)
x = layers.Conv2D(32, 7, strides=(1, 1), activation='relu', padding='same')(new_input)
x = layers.Conv2D(32, 7, strides=(1, 1), activation='relu', padding='same')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(64, 5, strides=(1, 1), activation='relu', padding='same')(x)
x = layers.Conv2D(64, 5, strides=(1, 1), activation='relu', padding='same')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(64, 3, strides=(1, 1), activation='relu', padding='same')(x)
x = layers.Conv2D(64, 3, strides=(1, 1), activation='relu', padding='same')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(64, 1, strides=(1, 1), activation='relu', padding='same')(x)
x = layers.Conv2D(64, 1, strides=(1, 1), activation='relu', padding='same')(x)
x = layers.BatchNormalization(axis=-1)(x)
FCN_output = layers.Conv2D(3, 1, strides=(1, 1), activation='relu', padding='same')(x)
FCN = keras.Model(new_input, FCN_output, name='FCN')
FCN.load_weights("FCN512_2_weights")
FCN.summary()

Model: "FCN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(2, 512, 512, 3)]        0         
_________________________________________________________________
conv2d_81 (Conv2D)           (2, 512, 512, 32)         4736      
_________________________________________________________________
conv2d_82 (Conv2D)           (2, 512, 512, 32)         50208     
_________________________________________________________________
batch_normalization_36 (Batc (2, 512, 512, 32)         128       
_________________________________________________________________
conv2d_83 (Conv2D)           (2, 512, 512, 64)         51264     
_________________________________________________________________
conv2d_84 (Conv2D)           (2, 512, 512, 64)         102464    
_________________________________________________________________
batch_normalization_37 (Batc (2, 512, 512, 64)         256     

In [63]:
# creates a dict of models/layers that can be easily accessed to get intermediate outputs
outputs_dict = dict([(layer.name, layer.output) for layer in FCN.layers[1:]]) #we wont need the output of input layer
outputs_dict

{'conv2d_81': <tf.Tensor 'conv2d_81/Relu:0' shape=(2, 512, 512, 32) dtype=float32>,
 'conv2d_82': <tf.Tensor 'conv2d_82/Relu:0' shape=(2, 512, 512, 32) dtype=float32>,
 'batch_normalization_36': <tf.Tensor 'batch_normalization_36/cond/Identity:0' shape=(2, 512, 512, 32) dtype=float32>,
 'conv2d_83': <tf.Tensor 'conv2d_83/Relu:0' shape=(2, 512, 512, 64) dtype=float32>,
 'conv2d_84': <tf.Tensor 'conv2d_84/Relu:0' shape=(2, 512, 512, 64) dtype=float32>,
 'batch_normalization_37': <tf.Tensor 'batch_normalization_37/cond/Identity:0' shape=(2, 512, 512, 64) dtype=float32>,
 'conv2d_85': <tf.Tensor 'conv2d_85/Relu:0' shape=(2, 512, 512, 64) dtype=float32>,
 'conv2d_86': <tf.Tensor 'conv2d_86/Relu:0' shape=(2, 512, 512, 64) dtype=float32>,
 'batch_normalization_38': <tf.Tensor 'batch_normalization_38/cond/Identity:0' shape=(2, 512, 512, 64) dtype=float32>,
 'conv2d_87': <tf.Tensor 'conv2d_87/Relu:0' shape=(2, 512, 512, 64) dtype=float32>,
 'conv2d_88': <tf.Tensor 'conv2d_88/Relu:0' shape=(2, 5

In [65]:
def get_GAP(layer_output):
    #input is a (height, width, channels) tensor. We first permute_dimensions to (channels, height, width). Then take the sum over all pixels in each channel.
    #each element j of the output is the mean across all pixels in the channel j
    return K.mean(K.batch_flatten(K.permute_dimensions(layer_output,(2,0,1))), axis=1)
    
GAP_target_img = []
GAP_learned_img = []

# cycle through all non-batch norm layers and concat the GAP into one long vector with length equal to the number of channels in the entire network
for name in outputs_dict:
    if name.find('batch_normalization') == -1: # skip batch norm layers
        layer_features = outputs_dict[name]
        GAP_target_img.append(get_GAP(layer_features[0]))
        GAP_learned_img.append(get_GAP(layer_features[1]))
    
GAP_target_img = K.concatenate(GAP_target_img, axis=0)
GAP_learned_img = K.concatenate(GAP_learned_img, axis=0)
GAP_learned_img

Loss = K.sqrt(K.sum(K.square(GAP_target_img-GAP_learned_img)))
grads = K.gradients(Loss, learned_image)[0]

print(GAP_target_img)
fetch_loss_and_grads = K.function([learned_image], [Loss, grads])

Tensor("concat_23:0", shape=(451,), dtype=float32)


In [66]:
class Evaluator():

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        #reshape optimizers vector into shape of input image
        x = x.reshape((1, img_height, img_width, 3)) #super long vector from optimizer being reshaped into a tensor. This is the learned image
        outs = fetch_loss_and_grads([x])#get loss and grads using this (current) img as input
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64') #reshape gradient tensor back into a vector for the optimizer
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values) #copy creates a new copy instead of a new reference name to the same object
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [67]:
# Run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the loss.
# Note that `scipy.optimize.fmin_l_bfgs_b` can only process flat vectors.
result_prefix = 'learned_image_05'
iterations = 1+15

initialcond_image_path = 'C:/Users/MrLin/Documents/Experiments/Fine_Scale_Autoencoder_Lichen/noiz.png'
x_vect = process_image(initialcond_image_path)
print(x_vect.shape)

x_vect = x_vect.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x_vect, min_val, info = fmin_l_bfgs_b(evaluator.loss, x_vect,
                                     fprime=evaluator.grads, maxfun=20)
    
    print('Current loss value:', min_val)
    if i%5==0:
        # Save current generated image
        img = x_vect.copy().reshape((img_height, img_width, 3)) #copy needed since we deprocess and we dont want to change the original x. since x will be used in next iteration
        img = deprocess_image(img)
        fname = result_prefix + '_at_iteration_%d.png' % i
        print(img.shape)
        pil_img = Image.fromarray(img)
        pil_img.save('C:/Users/MrLin/Documents/Experiments/Fine_Scale_Autoencoder_Lichen/Results/' + fname)

#         imsave(fname, img)
        end_time = time.time()
        print('Image saved as', fname)
        print('Iteration %d completed in %ds' % (i, end_time - start_time))

(1, 512, 512, 3)
Start of iteration 0
Current loss value: 6.424539
(512, 512, 3)
Image saved as learned_image_05_at_iteration_0.png
Iteration 0 completed in 5s
Start of iteration 1
Current loss value: 2.1694627
Start of iteration 2
Current loss value: 0.3884461
Start of iteration 3
Current loss value: 0.14136195
Start of iteration 4
Current loss value: 0.10618133
Start of iteration 5
Current loss value: 0.091603875
(512, 512, 3)
Image saved as learned_image_05_at_iteration_5.png
Iteration 5 completed in 4s
Start of iteration 6
Current loss value: 0.0856784
Start of iteration 7
Current loss value: 0.08242793
Start of iteration 8
Current loss value: 0.07973346
Start of iteration 9
Current loss value: 0.077586696
Start of iteration 10
Current loss value: 0.07404898
(512, 512, 3)
Image saved as learned_image_05_at_iteration_10.png
Iteration 10 completed in 4s
Start of iteration 11
Current loss value: 0.07141453
Start of iteration 12
Current loss value: 0.06758281
Start of iteration 13
Curr

Current loss value: 0.062284857
(512, 512, 3)
Image saved as learned_image_05_at_iteration_15.png
Iteration 15 completed in 4s
